In [1]:
import os
import sys
import json
from typing import List, Dict, Any, Optional, Tuple, Literal
from enum import Enum

In [2]:
os.getcwd()

'c:\\Users\\TempAccess\\Documents\\Dhruv\\RAG\\microsoft_graph_rag'

In [3]:
os.chdir('c:\\Users\\TempAccess\\Documents\\Dhruv\\RAG')
os.getcwd()

'c:\\Users\\TempAccess\\Documents\\Dhruv\\RAG'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from helper_function_openai import (
    Document,
    RetrievalResult,
    OpenAIEmbedder,
    FAISSVectorStore,
    OpenAIChat,
    read_pdf,
    chunk_text,
)

In [6]:
from openai import AzureOpenAI, OpenAI

In [7]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
oai = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
oai

In [9]:
climate_text = read_pdf(r"data\Understanding_Climate_Change.pdf")
climate_text

'Understanding Climate Change \nChapter 1: Introduction to Climate Change \nClimate change refers to significant, long-term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past century, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change. \nHistorical Context \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which \nbegan at the end of the last ice a

In [10]:
if not os.path.exists("data/climate_data.md"):
    with open("data/climate_data.md", "w", encoding="utf-8") as f:
        f.write(climate_text)

else:
    with open("data/climate_data.md", "r") as f:
        climate_text = f.read()

In [38]:
%pip install graphrag

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
climate_text

'Understanding Climate Change \nChapter 1: Introduction to Climate Change \nClimate change refers to significant, long-term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past century, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change. \nHistorical Context \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which \nbegan at the end of the last ice a

In [18]:
import yaml

In [19]:
GRAPHRAG_ROOT = 'data/graphrag'

In [20]:
# Initialize GraphRAG project if it doesn't exist
if not os.path.exists(GRAPHRAG_ROOT):
    !python -m graphrag init --root {GRAPHRAG_ROOT} --model gpt-4o-mini --embedding text-embedding-3-small
    print("GraphRAG project initialized")
else:
    print("GraphRAG project already exists")


GraphRAG project already exists


In [21]:
with open(os.path.join(GRAPHRAG_ROOT, '.env'), 'w') as f:
    f.write(f'GRAPHRAG_API_KEY={OPENAI_API_KEY}\n')
print("API key written to GraphRAG .env")


API key written to GraphRAG .env


In [22]:
with open(os.path.join(GRAPHRAG_ROOT, 'settings.yaml'), 'r') as f:
    settings = yaml.safe_load(f)

print(f"\nConfiguration:")
print(f"  LLM Model:       {settings['completion_models']['default_completion_model']['model']}")
print(f"  Embedding Model: {settings['embedding_models']['default_embedding_model']['model']}")
print(f"  Chunk Size:      {settings['chunking']['size']} tokens")
print(f"  Entity Types:    {settings['extract_graph']['entity_types']}")


Configuration:
  LLM Model:       gpt-4o-mini
  Embedding Model: text-embedding-3-small
  Chunk Size:      1200 tokens
  Entity Types:    ['organization', 'person', 'geo', 'event']


In [23]:
import shutil

input_dir = os.path.join(GRAPHRAG_ROOT, 'input')
os.makedirs(input_dir, exist_ok=True)

dest = os.path.join(input_dir, 'climate_data.txt')
if not os.path.exists(dest):
    shutil.copy('data/climate_data.md', dest)
    print(f"Copied document to {dest}")
else:
    print(f"Document already in place")

Document already in place


In [24]:
output_dir = f"{GRAPHRAG_ROOT}/output"

In [25]:
output_dir

'data/graphrag/output'

In [26]:
if not os.path.exists(output_dir) or not os.listdir(output_dir):
    print("Starting indexing... (this takes a few minutes)")
    print("=" * 50)
    !python -m graphrag index --root {GRAPHRAG_ROOT}
else:
    print("Index already exists — skipping. Delete output/ to re-index.")

Index already exists — skipping. Delete output/ to re-index.


In [27]:
print("Generated files:")
print("=" * 50)
for root, dirs, files in os.walk(os.path.join(GRAPHRAG_ROOT, 'output')):
    for f in files:
        filepath = os.path.join(root, f)
        size = os.path.getsize(filepath)
        rel = os.path.relpath(filepath, GRAPHRAG_ROOT)
        if size > 0:
            print(f"  {rel:60s} {size:>10,} bytes")

Generated files:
  output\communities.parquet                                       15,295 bytes
  output\community_reports.parquet                                109,957 bytes
  output\context.json                                                   2 bytes
  output\documents.parquet                                         43,525 bytes
  output\entities.parquet                                          40,164 bytes
  output\relationships.parquet                                     30,644 bytes
  output\stats.json                                                 2,405 bytes
  output\text_units.parquet                                        59,808 bytes
  output\lancedb\community_full_content.lance\data\4158ac8b-75e5-49c1-b847-316c2912c3c5.lance     94,649 bytes
  output\lancedb\community_full_content.lance\data\4a1e8fc2-9090-4eb9-b63f-1084f9a118d7.lance      6,709 bytes
  output\lancedb\community_full_content.lance\_indices\977f93b8-18a1-4d92-9a94-c4f96af0748e\auxiliary.idx      6,787 byte

In [28]:
GRAPHRAG_ROOT

'data/graphrag'

In [29]:
import subprocess
import re

def ask_graphrag(query, method="local"):
    """
    Query GraphRAG.
    
    Args:
        query:   Your question
        method:  'local' (entity-centric) or 'global' (holistic)
    
    Returns:
        Answer string
    """
    command = [
        sys.executable, '-m', 'graphrag', 'query',
        '--root', GRAPHRAG_ROOT,
        '--method', method,
        query,
    ]
    
    try:
        output = subprocess.check_output(
            command, 
            encoding='utf-8',
            errors='replace',
            universal_newlines=True, 
            stderr=subprocess.DEVNULL
        )
        
        # Parse the response
        if 'SUCCESS:' in output:
            response = output.split('SUCCESS:')[1].strip()
        elif 'Search Response:' in output:
            response = output.split('Search Response:')[1].strip()
        else:
            response = output.strip()
        
        # Remove [Data: ...] references
        response = re.sub(r'\[Data:.*?\]', '', response).strip()
        return response
        
    except subprocess.CalledProcessError as e:
        return f"Error: {e}"

print("Query functions ready")

Query functions ready


In [30]:
GRAPHRAG_ROOT

'data/graphrag'

In [31]:
from IPython.display import Markdown

local_query = "What are the main causes of climate change?"
print(f"LOCAL SEARCH: {local_query}\n")

local_result = ask_graphrag(local_query, method='local')
Markdown(local_result)

LOCAL SEARCH: What are the main causes of climate change?



## Main Causes of Climate Change

Climate change is primarily driven by human activities that alter the composition of the Earth's atmosphere. Understanding these causes is crucial for addressing the ongoing environmental crisis.

### Greenhouse Gas Emissions

One of the foremost contributors to climate change is the emission of greenhouse gases (GHGs). Activities such as burning fossil fuels (coal, oil, and natural gas) for energy production release significant amounts of carbon dioxide (CO2) and methane (CH4) into the atmosphere. These gases trap heat, creating the greenhouse effect. According to data, carbon emissions are a major cause of rising global temperatures, which have increased by approximately 1.2 degrees Celsius since the late 19th century as a result of these activities .

### Deforestation

Deforestation significantly contributes to climate change by reducing the number of trees that can absorb CO2, thereby diminishing the Earth's natural carbon sink. When forests are cut down or burned, the carbon stored in trees is released back into the atmosphere. This not only increases overall CO2 levels but also affects local and global ecosystems, leading to biodiversity loss and altering natural weather patterns .

### Agriculture

Agricultural practices also play a significant role in climate change. Traditional farming methods, particularly those that heavily rely on chemical fertilizers and pesticides, release nitrous oxide (N2O) and contribute to greenhouse gas emissions. Furthermore, livestock farming produces a large amount of methane through enteric fermentation, exacerbating the greenhouse effect and impacting climate stability .

### Industrial Processes

Industries contribute heavily to climate change through energy consumption and emissions of various pollutants. Manufacturing processes, especially in sectors like cement production, result in considerable CO2 emissions. The reliance on non-renewable energy sources in these processes perpetuates the cycle of greenhouse gas emissions, further driving climate change .

### Urbanization

Urbanization leads to increased energy demand and higher emissions due to the concentration of populations in cities, resulting in higher transportation emissions and energy consumption. Urban areas, often known for their significant carbon footprints, face challenges like heat islands and increased pollution, which further contribute to climate issues .

## Conclusion

The causes of climate change are interconnected and rooted in various human activities. Addressing these issues comprehensively requires a multi-faceted approach, including transitioning to renewable energy sources, implementing sustainable agricultural practices, and protecting forests. By understanding these underlying factors, societies can better equip themselves to combat climate change effectively .

In [32]:
local_result

"## Main Causes of Climate Change\n\nClimate change is primarily driven by human activities that alter the composition of the Earth's atmosphere. Understanding these causes is crucial for addressing the ongoing environmental crisis.\n\n### Greenhouse Gas Emissions\n\nOne of the foremost contributors to climate change is the emission of greenhouse gases (GHGs). Activities such as burning fossil fuels (coal, oil, and natural gas) for energy production release significant amounts of carbon dioxide (CO2) and methane (CH4) into the atmosphere. These gases trap heat, creating the greenhouse effect. According to data, carbon emissions are a major cause of rising global temperatures, which have increased by approximately 1.2 degrees Celsius since the late 19th century as a result of these activities .\n\n### Deforestation\n\nDeforestation significantly contributes to climate change by reducing the number of trees that can absorb CO2, thereby diminishing the Earth's natural carbon sink. When fo

In [44]:
import subprocess
import re

def ask_graphrag(query, method="local"):
    """
    Query GraphRAG.
    
    Args:
        query:   Your question
        method:  'local' (entity-centric) or 'global' (holistic)
    
    Returns:
        Answer string
    """
    command = [
        sys.executable, '-m', 'graphrag', 'query',
        '--root', GRAPHRAG_ROOT,
        '--method', method,
        query,
    ]
    
    result = subprocess.run(
        command,
        capture_output=True,
        text=True,
    )
    
    # If it failed, show the REAL error
    if result.returncode != 0:
        print("=== STDOUT ===")
        print(result.stdout[-500:] if result.stdout else "(empty)")
        print("=== STDERR ===")
        print(result.stderr[-500:] if result.stderr else "(empty)")
        
        return f"Failed with exit code {result.returncode}"
    
    output = result.stdout
    
    # Parse the response
    if 'SUCCESS:' in output:
        response = output.split('SUCCESS:')[1].strip()
    elif 'Search Response:' in output:
        response = output.split('Search Response:')[1].strip()
    else:
        response = output.strip()
    
    response = re.sub(r'\[Data:.*?\]', '', response).strip()
    return response


In [45]:
from IPython.display import Markdown

local_query = "What are the main causes of climate change?"
print(f"LOCAL SEARCH: {local_query}\n")

local_result = ask_graphrag(local_query, method='local')
Markdown(local_result)

LOCAL SEARCH: What are the main causes of climate change?



Climate change is primarily driven by significant shifts in weather patterns and temperatures, primarily due to human activities. The main causes of climate change can be summarized as follows:

### Human Activities
1. **Burning of Fossil Fuels**: The combustion of fossil fuels such as coal, oil, and natural gas for energy is the largest single source of greenhouse gas emissions. These activities release carbon dioxide (CO2) and other pollutants into the atmosphere, significantly contributing to the greenhouse effect, which traps heat and raises global temperatures .

2. **Deforestation**: The clearing of forests for agriculture, urban development, and other purposes reduces the number of trees that can absorb CO2 from the atmosphere. This loss of vegetation not only increases CO2 levels but also disrupts local ecosystems and diminishes biodiversity, making it a critical factor in climate change .

3. **Agricultural Practices**: Agricultural activities contribute to climate change through methane (CH4) emissions from livestock and belching, as well as nitrous oxide (N2O) emissions from fertilized soils. The agricultural sector is significantly affected by climate change, which alters crop production, pest control, and resource availability .

4. **Industrial Processes**: Industries often emit greenhouse gases as a byproduct, which includes CO2 from chemical reactions and energy consumption. The high resource consumption and waste generation in industrial operations contribute to environmental degradation and exacerbate climate issues .

### Natural Factors
While human activities are the primary drivers of contemporary climate change, natural factors also play a role, albeit to a lesser extent:

1. **Volcanic Eruptions**: These can release significant amounts of CO2 and ash into the atmosphere, leading to short-term climate changes. However, their contribution to long-term climate change is minimal compared to the effects of human activities .

2. **Solar Energy Variations**: Changes in solar radiation can influence climate patterns; however, these variations occur over long timescales and have a minor impact compared to human-induced changes .

### Conclusion
The consensus among scientists is clear: human-induced changes, particularly through fossil fuel consumption, deforestation, and industrial processes, are the principal causes of the ongoing climate crisis. Recognizing these causes is essential for developing effective strategies to mitigate climate change and transition to more sustainable practices .

In [33]:
from evaluation.evalute_rag import RAGEvaluator, create_test_cases
from generate_rag_report import RAGReportGenerator


In [36]:
class evaluate_rag_techniques:
    def __init__(self, file_path:str, question_file_path:str, metric_list:list[str]):
        self.file_path = file_path
        self.question_file_path = question_file_path
        self.questions = []
        self.ground_truth = []
        self.load_questions()
        self.metric_list = metric_list
        self.evaluator = RAGEvaluator()
    
    def load_questions(self):
        json_file = ""
        with open(self.question_file_path, 'r', encoding='utf-8') as f:
            json_file = json.load(f)
        
        for idx, data  in json_file.items():
            self.questions.append(data['question'])
            self.ground_truth.append(data['ground_truth'])


    def evaluate(self, pdf_output_path:str="rag_techniques_output.pdf", json_output_path:str="rag_techniques_output.json"):
        context_chunk_report = []
        hyde_report = []
        hyper_report = []
        query_transform_report = []
        reliable_report = []
        proposition_chunk_report = []
        simple_report = []

        output_dict = {}

        rag_llm_outputs = []
        # rag_llm_contexts = []
        for question, ground_truth in zip(self.questions, self.ground_truth):
            rag_llm_output = ask_graphrag(question, method='local')
            rag_llm_outputs.append(rag_llm_output)
            # rag_llm_contexts.append(rag_llm_context)

        rag_test_cases = create_test_cases(
            questions=self.questions,
            generated_answers=rag_llm_outputs,
            ground_truths=self.ground_truth,
        )

        rag_report = self.evaluator.evaluate_batch(rag_test_cases, metrics=self.metric_list)

        output_dict["microsoft_rag"] = json.dumps(rag_report.to_dict(), indent=4, default=str)

        
        with open(json_output_path, "w", encoding='utf-8') as f:
            json.dump(output_dict, f, indent=4, default=str)

        pdf_report_generator = RAGReportGenerator(json_input_filepath=json_output_path, pdf_output_filepath=pdf_output_path)
        pdf_report_generator.generate()

In [37]:
demo_evaluate_rag_techniques = evaluate_rag_techniques(file_path=r"data\IntermediaryGuidelinesandDigitalMediaEthicsCode.pdf", question_file_path=r"data\gold_standared_q_a.json", metric_list=["correctness", "faithfulness", "relevancy", "completeness"])

jsonfile_output_path = "microsoft_rag_techniques_output.json"
pdf_output_path = "microsoft_rag_techniques_output.pdf"

demo_evaluate_rag_techniques.evaluate(json_output_path=jsonfile_output_path, pdf_output_path=pdf_output_path)


[1/15] Evaluating: What is the definition of a “child” in these rules?...


2026-02-23 17:07:52,986 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:07:54,028 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:07:55,058 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:07:56,552 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: What is the definition of a “child” in these rules?

[2/15] Evaluating: What does “online real money game” mean?...


2026-02-23 17:07:57,536 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:07:58,493 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:07:59,606 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:08:01,194 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: What does “online real money game” mean?

[3/15] Evaluating: Within what time must an intermediary remove/disable unlawfu...


...

  [✗ correctness=0.30 | ✗ faithfulness=0.00 | ✓ relevancy=1.00 | ✗ completeness=0.25]  Q: What actions can a self-regulating body (Level II) issue aft

[14/15] Evaluating: Explain the verification lifecycle of a permissible online r...


2026-02-23 17:09:26,979 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:28,093 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:29,233 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:34,354 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: Explain the verification lifecycle of a permissible online r

[15/15] Evaluating: For “A” rated online curated content, what access restrictio...


2026-02-23 17:09:36,038 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:37,048 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:38,090 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:40,586 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-23 17:09:40,607 - INFO - Generating PDF report at microsoft_rag_techniques_output.pdf
2026-02-23 17:09:40,607 - INFO - Loading data from microsoft_rag_techniques_output.json
2026-02-23 17:09:40,629 - INFO -  identified winner: microsoft_rag with avg score: 0.3117
2026-02-23 17:09:40,714 - INFO - PDF Generation Complete


  [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✓ relevancy=1.00 | ✗ completeness=0.00]  Q: For “A” rated online curated content, what access restrictio

  RAG EVALUATION REPORT
  Model: gpt-4o | Cases: 15 | Time: 108.5s
  1. [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: What is the definition of a “child” in these rules?
  2. [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: What does “online real money game” mean?
  3. [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: Within what time must an intermediary remove/disable unlawfu
  4. [✗ correctness=0.00 | ✗ faithfulness=0.00 | ✓ relevancy=1.00 | ✗ completeness=0.00]  Q: What are the two ways “actual knowledge” can arise (as per R
  5. [✗ correctness=0.00 | ✓ faithfulness=1.00 | ✗ relevancy=0.00 | ✗ completeness=0.00]  Q: What are the content rating categories for online curated co
  6. [✗ correctness=0.00 | ✓ faithfulnes